# CE49X: Introduction to Computational Thinking and Data Science for Civil Engineers
## Week 6 — Theory: Model Evaluation Philosophy

**Instructor:** Dr. Eyuphan Koc  
**Department of Civil Engineering, Bogazici University**  
**Semester:** Spring 2026

*Companion notebook to Week 6 lecture — designed for self-study and in-class discussion*

---

## Table of Contents

1. [The Accuracy Trap](#1.-The-Accuracy-Trap)
2. [The Confusion Matrix](#2.-The-Confusion-Matrix)
3. [Precision, Recall, and the Tradeoff](#3.-Precision,-Recall,-and-the-Tradeoff)
4. [ROC Curves and AUC](#4.-ROC-Curves-and-AUC)
5. [Overfitting — When Your Model Memorizes](#5.-Overfitting-—-When-Your-Model-Memorizes)
6. [Data Leakage — The Silent Killer](#6.-Data-Leakage-—-The-Silent-Killer)
7. [Putting It Together](#7.-Putting-It-Together)

---

## 1. The Accuracy Trap

Every beginner's first question about a model is: *"What's the accuracy?"*

This section shows why that question, on its own, can be dangerously misleading — especially in civil engineering applications where failures are rare but catastrophic.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, confusion_matrix, classification_report,
                             precision_recall_curve, roc_curve, auc, 
                             mean_squared_error, r2_score)
%matplotlib inline

In [ ]:
np.random.seed(42)

# Structural health monitoring dataset — HIGHLY IMBALANCED
n_safe = 990
n_damaged = 10

# Features: vibration_freq, max_deflection, crack_width_index
X_safe = np.column_stack([
    np.random.normal(5.0, 0.5, n_safe),    # vibration frequency (Hz)
    np.random.normal(2.0, 0.3, n_safe),    # max deflection (mm)
    np.random.normal(0.1, 0.05, n_safe)    # crack width index
])
X_damaged = np.column_stack([
    np.random.normal(3.5, 0.8, n_damaged),
    np.random.normal(4.0, 0.5, n_damaged),
    np.random.normal(0.5, 0.1, n_damaged)
])

X = np.vstack([X_safe, X_damaged])
y = np.array([0] * n_safe + [1] * n_damaged)  # 0=safe, 1=damaged

print(f"Dataset: {len(X)} bridges")
print(f"  Safe:    {n_safe} ({n_safe/len(X)*100:.1f}%)")
print(f"  Damaged: {n_damaged} ({n_damaged/len(X)*100:.1f}%)")

In [ ]:
# A model that ALWAYS predicts "safe"
class AlwaysSafeModel:
    def predict(self, X):
        return np.zeros(len(X))

dumb_model = AlwaysSafeModel()
y_pred_dumb = dumb_model.predict(X)
accuracy_dumb = accuracy_score(y, y_pred_dumb)
print(f"Accuracy of the 'always safe' model: {accuracy_dumb:.1%}")
print(f"\nBut how many damaged bridges did it catch?")
print(f"Damaged bridges detected: {np.sum(y_pred_dumb[y == 1]):.0f} out of {n_damaged}")

> **Key Insight: Accuracy is Meaningless When Classes are Imbalanced**
> The model above has 99% accuracy — and it's **completely useless**. It missed ALL 10 damaged bridges.
>
> In engineering, failures are rare (thankfully). But that makes accuracy a terrible metric for safety-critical applications.

[DISCUSS] Can you think of other engineering scenarios where the "rare event" is the one you care about most?

---

## 2. The Confusion Matrix

The confusion matrix tells us **how** the model is wrong — not just whether it's wrong. This distinction matters enormously when different types of errors have different consequences.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

model = LogisticRegression(random_state=42, class_weight='balanced')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(7, 6))
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['Predicted Safe', 'Predicted Damaged'], fontsize=11)
ax.set_yticklabels(['Actually Safe', 'Actually Damaged'], fontsize=11)
for i in range(2):
    for j in range(2):
        label = f'{cm[i, j]}'
        if i == 0 and j == 0: label += '\n(TN)'
        elif i == 0 and j == 1: label += '\n(FP)'
        elif i == 1 and j == 0: label += '\n(FN)'
        else: label += '\n(TP)'
        ax.text(j, i, label, ha='center', va='center', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax)
ax.set_title('Confusion Matrix — Bridge Safety Classifier', fontsize=13)
plt.tight_layout()
plt.show()

> **Definition: Confusion Matrix Quadrants**
>
> | | Predicted Negative | Predicted Positive |
> |---|---|---|
> | **Actually Negative** | True Negative (TN) | False Positive (FP) |
> | **Actually Positive** | False Negative (FN) | True Positive (TP) |
>
> - **TN**: Correctly identified as safe
> - **FP**: Safe bridge flagged as damaged ("false alarm")
> - **FN**: Damaged bridge missed ("missed detection")
> - **TP**: Correctly identified as damaged

> **Key Insight: In Civil Engineering, Errors are NOT Symmetric**
>
> | Error Type | Meaning | Consequence |
> |---|---|---|
> | **False Negative (FN)** | Damaged bridge predicted as safe | **People could die** |
> | **False Positive (FP)** | Safe bridge predicted as damaged | Unnecessary inspection (costs money) |
>
> A false negative in structural health monitoring can be catastrophic. A false positive is merely expensive.

In [ ]:
# Two models, same accuracy (~95%), very different confusion matrices
print("=== Model A ===")
print("Accuracy: 95.0%")
print("  TN=285  FP=12")
print("  FN=3    TP=0")
print("  → Caught 0 out of 3 damaged bridges")
print()
print("=== Model B ===")
print("Accuracy: 95.0%")
print("  TN=282  FP=15")
print("  FN=0    TP=3")
print("  → Caught ALL 3 damaged bridges (with 15 false alarms)")
print()
print("[DISCUSS] Which model would you deploy on a real bridge?")

In [ ]:
# [QUICK] Full classification report from sklearn
print("Classification Report — Bridge Safety Classifier")
print("=" * 55)
print(classification_report(y_test, y_pred, target_names=['Safe', 'Damaged']))

---

## 3. Precision, Recall, and the Tradeoff

Now that we understand the confusion matrix, we can define two critical metrics:

> **Definition: Precision and Recall**
>
> **Precision** = TP / (TP + FP) — *"Of all alarms raised, how many were real?"*
>
> **Recall** = TP / (TP + FN) — *"Of all actual positives, how many did we catch?"*
>
> These two metrics are in tension. Improving one typically hurts the other.

In [ ]:
y_proba = model.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.1, 0.95, 0.05)
precisions = []
recalls = []

for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)
    tp = np.sum((y_pred_t == 1) & (y_test == 1))
    fp = np.sum((y_pred_t == 1) & (y_test == 0))
    fn = np.sum((y_pred_t == 0) & (y_test == 1))
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0
    precisions.append(prec)
    recalls.append(rec)

# Show a few key thresholds
print(f"{'Threshold':>10} {'Precision':>10} {'Recall':>10} {'Interpretation':>40}")
print("-" * 75)
for i in range(0, len(thresholds), 3):
    t = thresholds[i]
    p = precisions[i]
    r = recalls[i]
    interp = "high recall (catch more)" if t < 0.3 else ("balanced" if t < 0.6 else "high precision (fewer false alarms)")
    print(f"{t:>10.2f} {p:>10.2f} {r:>10.2f} {interp:>40}")

In [ ]:
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_proba)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(recall_curve, precision_curve, 'b-', linewidth=2)
ax.set_xlabel('Recall (How many damaged bridges we catch)', fontsize=12)
ax.set_ylabel('Precision (How many alarms are real)', fontsize=12)
ax.set_title('Precision-Recall Tradeoff', fontsize=13)
ax.grid(True, alpha=0.3)
ax.set_xlim([0, 1.05])
ax.set_ylim([0, 1.05])
plt.tight_layout()
plt.show()

> **Key Insight: You Can't Maximize Both**
> - **Optimize Recall** when missing a positive is catastrophic → structural safety monitoring
> - **Optimize Precision** when false alarms are costly → routine maintenance scheduling
> - **F1 Score** = harmonic mean of precision and recall — use when you want one number that balances both

> **Example: Civil Engineering Applications**
> - **Structural safety** → High recall (don't miss damage, even if you get some false alarms)
> - **Routine maintenance** → High precision (don't waste inspections on false alarms)
> - **Emergency response** → High recall (better safe than sorry)

In [ ]:
# [PRACTICE] Compute F1 score manually
# F1 = 2 * (precision * recall) / (precision + recall)

tp = np.sum((y_pred == 1) & (y_test == 1))
fp = np.sum((y_pred == 1) & (y_test == 0))
fn = np.sum((y_pred == 0) & (y_test == 1))

precision_val = tp / (tp + fp) if (tp + fp) > 0 else 0
recall_val = tp / (tp + fn) if (tp + fn) > 0 else 0
f1_val = 2 * (precision_val * recall_val) / (precision_val + recall_val) if (precision_val + recall_val) > 0 else 0

print(f"Precision: {precision_val:.3f}")
print(f"Recall:    {recall_val:.3f}")
print(f"F1 Score:  {f1_val:.3f}")
print(f"\n[TOGETHER] What does this F1 score tell us about the model's balance?")

---

## 4. ROC Curves and AUC

The **Receiver Operating Characteristic (ROC)** curve gives us a way to evaluate a classifier across **all possible thresholds** simultaneously. The **Area Under the Curve (AUC)** summarizes this into a single number.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Train multiple models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, class_weight='balanced'),
    'Decision Tree': DecisionTreeClassifier(random_state=42, class_weight='balanced'),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5)
}

fig, ax = plt.subplots(figsize=(8, 7))
ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Guessing (AUC=0.50)')

for name, m in models.items():
    m.fit(X_train, y_train)
    if hasattr(m, 'predict_proba'):
        y_score = m.predict_proba(X_test)[:, 1]
    else:
        y_score = m.decision_function(X_test)
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, linewidth=2, label=f'{name} (AUC={roc_auc:.2f})')

ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curves — Comparing Models', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

> **Key Insight: AUC Tells You How Well Your Model Separates Classes**
> - AUC = 1.0: Perfect separation
> - AUC = 0.5: Random guessing (the diagonal)
> - Higher AUC = better model, regardless of threshold choice

> **Definition: ROC Curve Axes**
>
> - **X-axis (False Positive Rate)** = FP / (FP + TN) — *"Of all safe bridges, how many did we falsely flag?"*
> - **Y-axis (True Positive Rate = Recall)** = TP / (TP + FN) — *"Of all damaged bridges, how many did we catch?"*
>
> A perfect model hugs the **top-left corner**. A random model lies along the **diagonal**.

In [ ]:
# [QUICK] Compare AUC values numerically
print("Model Comparison Summary")
print("=" * 50)
for name, m in models.items():
    y_pred_m = m.predict(X_test)
    if hasattr(m, 'predict_proba'):
        y_score_m = m.predict_proba(X_test)[:, 1]
    else:
        y_score_m = m.decision_function(X_test)
    fpr_m, tpr_m, _ = roc_curve(y_test, y_score_m)
    roc_auc_m = auc(fpr_m, tpr_m)
    acc_m = accuracy_score(y_test, y_pred_m)
    print(f"{name:30s}  Accuracy={acc_m:.3f}  AUC={roc_auc_m:.3f}")

print("\n[DISCUSS] Which model would you choose and why?")
print("Hint: AUC is often more informative than accuracy for imbalanced problems.")

---

## 5. Overfitting — When Your Model Memorizes

A model that performs perfectly on training data but poorly on new data has **memorized** the noise in the training set rather than learning the underlying pattern. This is one of the most fundamental challenges in machine learning.

In [ ]:
np.random.seed(42)
n_pts = 30
x = np.sort(np.random.uniform(0, 10, n_pts))
y_true = np.sin(x)
y_noisy = y_true + np.random.normal(0, 0.3, n_pts)

fig, axes = plt.subplots(1, 4, figsize=(20, 4))
x_plot = np.linspace(0, 10, 300)

for ax, degree in zip(axes, [1, 3, 10, 20]):
    coeffs = np.polyfit(x, y_noisy, degree)
    y_fit = np.polyval(coeffs, x_plot)
    
    ax.scatter(x, y_noisy, s=40, color='steelblue', edgecolors='k', zorder=3)
    ax.plot(x_plot, y_fit, 'r-', linewidth=2)
    ax.plot(x_plot, np.sin(x_plot), 'g--', linewidth=1, alpha=0.5, label='True function')
    ax.set_title(f'Degree {degree}', fontsize=12)
    ax.set_ylim(-2, 2)
    ax.grid(True, alpha=0.3)
    if degree == 1: ax.legend(fontsize=9)

plt.suptitle('Polynomial Fits: From Underfitting to Overfitting', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

> **Definition: Underfitting vs. Overfitting**
>
> - **Underfitting** (degree 1): Model is too simple to capture the pattern. High bias, low variance.
> - **Just right** (degree 3): Model captures the pattern without fitting the noise.
> - **Overfitting** (degree 20): Model memorizes the noise. Low bias, high variance.
>
> The goal is to find the **sweet spot** — complex enough to capture real patterns, simple enough to generalize.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# Split data
x_train_ov, x_test_ov, y_train_ov, y_test_ov = train_test_split(
    x, y_noisy, test_size=0.3, random_state=42
)

degrees = range(1, 16)
train_errors = []
test_errors = []

for d in degrees:
    pipe = make_pipeline(PolynomialFeatures(d), LinearRegression())
    pipe.fit(x_train_ov.reshape(-1, 1), y_train_ov)
    
    train_pred = pipe.predict(x_train_ov.reshape(-1, 1))
    test_pred = pipe.predict(x_test_ov.reshape(-1, 1))
    
    train_errors.append(np.sqrt(mean_squared_error(y_train_ov, train_pred)))
    test_errors.append(np.sqrt(mean_squared_error(y_test_ov, test_pred)))

# Cap very large test errors for visualization
test_errors_capped = [min(e, 5) for e in test_errors]

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(list(degrees), train_errors, 'o-', color='steelblue', linewidth=2, label='Training Error (RMSE)')
ax.plot(list(degrees), test_errors_capped, 's-', color='indianred', linewidth=2, label='Test Error (RMSE)')
ax.set_xlabel('Polynomial Degree', fontsize=12)
ax.set_ylabel('RMSE', fontsize=12)
ax.set_title('Train vs Test Error: The Overfitting Curve', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.axvline(x=3, color='green', linestyle='--', alpha=0.5, label='Sweet spot')
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

> **Key Insight: A Model That's Perfect on Training Data is Suspicious, Not Impressive**
> - Training error always decreases with complexity
> - Test error decreases, then **increases** (the U-curve)
> - The gap between train and test error = overfitting

> **Example: Civil Engineering**
> If your concrete strength model fits your lab data perfectly but fails on site data, it memorized the lab conditions instead of learning the physics.

In [ ]:
# [TOGETHER] Let's look at the numbers
print(f"{'Degree':>8} {'Train RMSE':>12} {'Test RMSE':>12} {'Gap':>10} {'Status':>15}")
print("-" * 60)
for d, tr_e, te_e in zip(degrees, train_errors, test_errors):
    gap = te_e - tr_e
    if gap < 0.1:
        status = "underfitting"
    elif gap < 0.5:
        status = "good fit"
    elif gap < 2.0:
        status = "overfitting"
    else:
        status = "SEVERE overfit"
    te_display = min(te_e, 99.99)
    print(f"{d:>8d} {tr_e:>12.4f} {te_display:>12.4f} {gap:>10.4f} {status:>15}")

---

## 6. Data Leakage — The Silent Killer

Data leakage occurs when information from outside the training dataset is used to create the model. It leads to **overly optimistic performance estimates** that collapse in production.

This is arguably the most common and most dangerous mistake in applied machine learning.

In [ ]:
np.random.seed(42)
n = 200

# Concrete mix data
cement = np.random.uniform(280, 400, n)
water_cement = np.random.uniform(0.35, 0.60, n)
strength_7d = 10 + 0.05 * cement - 15 * water_cement + np.random.normal(0, 2, n)
strength_28d = 1.3 * strength_7d + 5 + np.random.normal(0, 3, n)

# Create a "leaked" feature — strength category derived FROM the 28-day target
strength_category = np.where(strength_28d > 35, 2, np.where(strength_28d > 25, 1, 0))

df_leak = pd.DataFrame({
    'cement_kg_m3': cement,
    'water_cement': water_cement,
    'strength_7d': strength_7d,
    'strength_category': strength_category,  # LEAKED!
    'strength_28d': strength_28d  # TARGET
})

print("Features available:")
print(df_leak.columns.tolist())
print("\n'strength_category' was derived FROM strength_28d — this is data leakage!")

In [ ]:
# WITH leakage
X_leaked = df_leak[['cement_kg_m3', 'water_cement', 'strength_7d', 'strength_category']].values
y_target = df_leak['strength_28d'].values
X_tr, X_te, y_tr, y_te = train_test_split(X_leaked, y_target, test_size=0.2, random_state=42)
model_leaked = LinearRegression().fit(X_tr, y_tr)

# WITHOUT leakage
X_clean = df_leak[['cement_kg_m3', 'water_cement', 'strength_7d']].values
X_tr2, X_te2, y_tr2, y_te2 = train_test_split(X_clean, y_target, test_size=0.2, random_state=42)
model_clean = LinearRegression().fit(X_tr2, y_tr2)

print(f"WITH leaked feature:    R² = {model_leaked.score(X_te, y_te):.3f}  <- Too good to be true!")
print(f"WITHOUT leaked feature: R² = {model_clean.score(X_te2, y_te2):.3f}  <- The real performance")
print(f"\nThe leaked model looks amazing but would fail in production,")
print(f"because 'strength_category' wouldn't be available at prediction time.")

> **Key Insight: Data Leakage Gives You False Confidence**
> Always ask: *"Would I have this feature at prediction time?"*
>
> **3 Common Leakage Patterns:**
> 1. **Target leakage**: A feature derived from the target variable
> 2. **Train-test contamination**: Scaling/normalizing before splitting
> 3. **Temporal leakage**: Using future data to predict the past

In [ ]:
# [PRACTICE] Train-test contamination example
from sklearn.preprocessing import StandardScaler

# WRONG way: scale before split
scaler_wrong = StandardScaler()
X_scaled_wrong = scaler_wrong.fit_transform(X_clean)  # fit on ALL data
Xw_tr, Xw_te, yw_tr, yw_te = train_test_split(X_scaled_wrong, y_target, test_size=0.2, random_state=42)
model_wrong = LinearRegression().fit(Xw_tr, yw_tr)

# RIGHT way: scale after split
Xr_tr, Xr_te, yr_tr, yr_te = train_test_split(X_clean, y_target, test_size=0.2, random_state=42)
scaler_right = StandardScaler()
Xr_tr_scaled = scaler_right.fit_transform(Xr_tr)     # fit ONLY on training
Xr_te_scaled = scaler_right.transform(Xr_te)          # transform test with train stats
model_right = LinearRegression().fit(Xr_tr_scaled, yr_tr)

print("Train-Test Contamination Demo")
print("=" * 45)
print(f"WRONG (scale before split): R² = {model_wrong.score(Xw_te, yw_te):.4f}")
print(f"RIGHT (scale after split):  R² = {model_right.score(Xr_te_scaled, yr_te):.4f}")
print(f"\nIn this case the difference is small, but with more features")
print(f"and less data, contamination can drastically inflate scores.")

> **Example: Civil Engineering**
>
> Imagine you are predicting 28-day concrete compressive strength at mix design time.
>
> - **Available at prediction time**: cement content, water-cement ratio, aggregate size, admixture type
> - **NOT available at prediction time**: 7-day strength (you'd need to wait 7 days!), slump test results from the pour
>
> Including 7-day strength as a feature is only valid if you explicitly frame the problem as "predict 28-day from 7-day results."
> The key is **clarity about when the prediction needs to happen**.

---

## 7. Putting It Together

### Model Evaluation Checklist

Before you report results, ask yourself:

| # | Question | If "No", Then... |
|---|---|---|
| 1 | Is my dataset balanced? | Accuracy is misleading — use precision, recall, F1 |
| 2 | Do I know the cost of each error type? | Define: What's worse, FP or FN? |
| 3 | Am I evaluating on truly unseen data? | Use train-test split or cross-validation |
| 4 | Does my model generalize? | Check train vs test error gap |
| 5 | Could there be data leakage? | Audit every feature: would I have it at prediction time? |

> **Key Insight: Model Evaluation is Not a Step — It's a Mindset**
> Evaluation pervades the **entire** workflow. From data collection (is this representative?) to deployment (is the model still accurate on new data?).

In [ ]:
# [TOGETHER] Summary quiz
print("Quick Self-Check")
print("=" * 50)
print()
print("Q1: Your earthquake damage classifier has 98% accuracy.")
print("    Should you celebrate? Why or why not?")
print()
print("Q2: Your model has perfect R² on training data.")
print("    What should you suspect?")
print()
print("Q3: You're predicting landslide risk. Which is worse:")
print("    predicting 'safe' when it's dangerous (FN), or")
print("    predicting 'dangerous' when it's safe (FP)?")
print()
print("Q4: Your concrete strength model uses 'quality_grade'")
print("    as a feature. This grade was assigned AFTER the")
print("    28-day test. Is this a problem?")

### Connection to the CE49X Course

This notebook gives you the evaluation framework for:
- **Week 07** (Naive Bayes): Before reporting "my Naive Bayes has 94% accuracy," ask — *accuracy at what cost?*
- **Week 08** (SVM): Before choosing an SVM kernel, check — *am I overfitting to the training set?*

> **Key Insight: The Right Question**
> Don't ask: "What's my model's accuracy?"
> Ask: "What's the cost of my model's errors, and can I live with it?"

---

### Questions?

**Dr. Eyuphan Koc**  
eyuphan.koc@bogazici.edu.tr